In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm # https://github.com/tqdm/tqdm
tqdm.pandas()

## Preprocessing

In this notebook process the raw files, which can be found in the `hhblock_dataset` folder. Each row in a block has a date, and along the columns, half-hourly blocks (compact form). We reshape this into long form so that each row has a date and a single half-hourly measurement (expanded form). We then combine the energy data with weather and bank holiday data to aid downstream analysis.

First, we read read and combine the rows from all blocks into a single dataframe:

In [2]:
source_data = Path('../data/')
block_data_path = source_data/'hhblock_dataset'

Some LCLid columns in the dataset end much earlier than the rest. This could be because particular households dropped out of the program early, moved house (leaving the house unoccupied) and so on. Because of this, We need to find the global latest date across all data blocks:

In [5]:
max_date = None
for f in tqdm(block_data_path.glob('*.csv')):
    df = pd.read_csv(f, parse_dates=False)
    df['day'] = pd.to_datetime(df['day'], yearfirst=True)
    if max_date is None:
        max_date = df['day'].max()
    else:
        if df['day'].max()>max_date:
            max_date = df['day'].max()
print(f'Max Date across all blocks: {max_date}')
del df

0it [00:00, ?it/s]

Max Date across all blocks: 2014-02-27 00:00:00


We define a function for converting the time series data into compact form:

In [9]:
def preprocess_compact(x, max_date):
    """Takes a DataFrame with time series data in compact form and a maximum date. 
    Returns the time series array, along with the time series identifier, start date,
    and the length of the series."""

    start_date = x['day'].min()
    name = x['LCLid'].unique()[0]
    ### Fill missing dates with NaN ###
    # Create a date range from  min to max
    dr = pd.date_range(start=x['day'].min(), end=max_date, freq='1D')
    # Add hh_0 to hh_47 to columns (for the 48 half-hourly measurements)
    dr = pd.DataFrame(columns=[f"hh_{i}" for i in range(48)], index=dr).unstack().reset_index()
    # renaming the columns
    dr.columns = ['hour_block', 'day', '_']
    # left merging the dataframe to the standard dataframe
    dr = dr.merge(x, on=['hour_block','day'], how='left') # missing values will be left as NaN
    # sorting the rows
    dr.sort_values(['day', 'offset'], inplace=True)
    # extracting the timeseries array
    ts = dr['energy_consumption'].values
    len_ts = len(ts)
    return start_date, name, ts, len_ts

In [10]:
def load_process_block_compact(block_df, max_date, freq='30min', 
                               ts_identifier='series_name', value_name='series_value'):
    """Takes a DataFrame for one of the blocks, a frequency, a time series identifier and 
    a value name. Applies compact processing to each of the LCLids (households) in the block.
    Returns the data for that block, converted into compact form."""
    grps = block_df.groupby('LCLid')
    all_series = []
    all_start_dates = []
    all_names = []
    all_data = {}
    all_len = []
    for idx, df in tqdm(grps, leave=False):
        start_date, name, ts, len_ts = preprocess_compact(df, max_date)
        all_series.append(ts)
        all_start_dates.append(start_date)
        all_names.append(name)
        all_len.append(len_ts)

    all_data[ts_identifier] = all_names
    all_data['start_timestamp'] = all_start_dates
    all_data['frequency'] = freq
    all_data[value_name] = all_series
    all_data['series_length'] = all_len
    return pd.DataFrame(all_data)

So that we can combine the energy consumption data with the weather and bank holiday data, we combine all block data into single data frame. To do this, we:

1) create a standard DataFrame using the start date and global end date (`max_date`)
2) left merge the DataFrame for each LCLid onto the standard DataFrame, leaving missing values as `np.nan`

In [ ]:
block_df_l = []

# wrapping in tqdm shows a smart progress meter as blocks are processed
# glob matches all .csv files in the block_data_path directory
for file in tqdm(sorted(list(block_data_path.glob('*.csv'))), desc='processing blocks...'):
    block_df = pd.read_csv(file, parse_dates=False)
    block_df['day'] = pd.to_datetime(block_df['day'], yearfirst=True)
    # take only from 2012-01-01
    block_df = block_df.loc[block_df['day']>='2012-01-01']
    # reshape the dataframe into long form with hour blocks along the rows
    block_df = block_df.set_index(['LCLid', 'day']).stack().reset_index().rename(columns={'level_2': 'hour_block', 0: 'energy_consumption'})
    # creating a numerical hourblock column
    block_df['offset'] = block_df['hour_block'].str.replace("hh_", "").astype(int)
    block_df_l.append(load_process_block_compact(block_df, max_date, freq='30min', ts_identifier='LCLid', value_name='energy_consumption'))

In [13]:
hhblock_df = pd.concat(block_df_l)
del block_df_l

In [14]:
hhblock_df.head()

,LCLid,start_timestamp,frequency,energy_consumption,series_length
0,MAC000002,2012-10-13,30min,"[0.263, 0.2689999999999999, 0.275, 0.256, 0.21...",24144
1,MAC000246,2012-01-01,30min,"[0.509, 0.317, 0.253, 0.249, 0.93, 0.607, 0.10...",37872
2,MAC000450,2012-03-23,30min,"[1.337, 1.426, 0.996, 0.971, 0.994, 0.952, 0.8...",33936
3,MAC001074,2012-05-09,30min,"[0.18, 0.086, 0.106, 0.173, 0.146, 0.223, 0.21...",31680
4,MAC003223,2012-09-18,30min,"[0.076, 0.079, 0.123, 0.109, 0.051, 0.069, 0.0...",25344


`informations_households.csv` contains metadata describing each household. Because these are not time-dependent (static) we left merge these on LCLid:

In [15]:
household_info = pd.read_csv(source_data/'informations_households.csv')
household_info.head()

,LCLid,stdorToU,Acorn,Acorn_grouped,file
0,MAC005492,ToU,ACORN-,ACORN-,block_0
1,MAC001074,ToU,ACORN-,ACORN-,block_0
2,MAC000002,Std,ACORN-A,Affluent,block_0
3,MAC003613,Std,ACORN-A,Affluent,block_0
4,MAC003597,Std,ACORN-A,Affluent,block_0


In [16]:
hhblock_df = hhblock_df.merge(household_info, on='LCLid', validate='one_to_one')

In [17]:
hhblock_df.head()

,LCLid,start_timestamp,frequency,energy_consumption,series_length,stdorToU,Acorn,Acorn_grouped,file
0,MAC000002,2012-10-13,30min,"[0.263, 0.2689999999999999, 0.275, 0.256, 0.21...",24144,Std,ACORN-A,Affluent,block_0
1,MAC000246,2012-01-01,30min,"[0.509, 0.317, 0.253, 0.249, 0.93, 0.607, 0.10...",37872,Std,ACORN-A,Affluent,block_0
2,MAC000450,2012-03-23,30min,"[1.337, 1.426, 0.996, 0.971, 0.994, 0.952, 0.8...",33936,Std,ACORN-A,Affluent,block_0
3,MAC001074,2012-05-09,30min,"[0.18, 0.086, 0.106, 0.173, 0.146, 0.223, 0.21...",31680,ToU,ACORN-,ACORN-,block_0
4,MAC003223,2012-09-18,30min,"[0.076, 0.079, 0.123, 0.109, 0.051, 0.069, 0.0...",25344,Std,ACORN-A,Affluent,block_0


`uk_bank_holidays.csv` describes dates of uk holidays. Energy consumption patterns might differ when households are spending more time in the home.

In [19]:
bank_holidays = pd.read_csv(source_data/'uk_bank_holidays.csv', parse_dates=False)
bank_holidays['Bank holidays'] = pd.to_datetime(bank_holidays['Bank holidays'], yearfirst=True)
bank_holidays.set_index('Bank holidays', inplace=True)
bank_holidays.head()

,Type
Bank holidays,
2012-12-26,Boxing Day
2012-12-25,Christmas Day
2012-08-27,Summer bank holiday
2012-05-06,Queen?s Diamond Jubilee (extra bank holiday)
2012-04-06,Spring bank holiday (substitute day)


In [20]:
# reindex on standard date range
# resample the holiday data to every 30 minutes
# forward fill holidays within a day, fill NaN values with NO_HOLIDAY
bank_holidays = bank_holidays.resample('30min').asfreq()
bank_holidays = bank_holidays.groupby(bank_holidays.index.date).ffill().fillna('NO_HOLIDAY')
bank_holidays.index.name='datetime'
bank_holidays.head()

,Type
datetime,
2012-02-01 00:00:00,New Year?s Day (substitute day)
2012-02-01 00:30:00,New Year?s Day (substitute day)
2012-02-01 01:00:00,New Year?s Day (substitute day)
2012-02-01 01:30:00,New Year?s Day (substitute day)
2012-02-01 02:00:00,New Year?s Day (substitute day)


`weather_hourly_darksky.csv` contains the hourly data from the darksky weather API. We upsample this to every 30 minutes and forward-fill missing values.

In [21]:
weather_hourly = pd.read_csv(source_data/'weather_hourly_darksky.csv', parse_dates=False)
weather_hourly['time'] = pd.to_datetime(weather_hourly['time'], yearfirst=True)
weather_hourly.set_index('time', inplace=True)
weather_hourly.head()

,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
time,,,,,,,,,,,
2011-11-11 00:00:00,5.97,104,10.24,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy
2011-11-11 01:00:00,4.88,99,9.76,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy
2011-11-11 02:00:00,3.70,98,9.46,8.79,1016.36,7.76,3.17,rain,partly-cloudy-night,0.96,Partly Cloudy
2011-11-11 03:00:00,3.12,99,9.23,8.63,1016.28,7.44,3.25,rain,fog,0.96,Foggy
2011-11-11 04:00:00,1.85,111,9.26,9.21,1015.98,7.24,3.70,rain,fog,1.00,Foggy


In [16]:
# resample the data to every 30 minutes, forward fill
weather_hourly = weather_hourly.resample("30min").ffill()
weather_hourly.head()

,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
time,,,,,,,,,,,
2011-11-01 00:00:00,13.63,160,13.49,11.48,1008.14,13.49,3.11,rain,clear-night,0.88,Clear
2011-11-01 00:30:00,13.63,160,13.49,11.48,1008.14,13.49,3.11,rain,clear-night,0.88,Clear
2011-11-01 01:00:00,13.26,154,12.73,11.58,1007.88,12.73,3.08,rain,partly-cloudy-night,0.93,Partly Cloudy
2011-11-01 01:30:00,13.26,154,12.73,11.58,1007.88,12.73,3.08,rain,partly-cloudy-night,0.93,Partly Cloudy
2011-11-01 02:00:00,12.94,161,13.65,12.14,1007.09,13.65,3.71,rain,clear-night,0.91,Clear


Here, as a nicety, we define a function for mapping the relevant holiday and weather data onto each row so that we can use `progress_apply` to complete our DataFrame incrementally.

In [23]:
def map_holidays_weather(row, bank_holidays, weather_hourly):
    """Takes a row from the (compact) energy dataset, a DataFrame of bank holidays and
    a DataFrame of hourly weather data.
    Returns the row with the appropriate weather and energy data joined on."""
    date_range = pd.date_range(row['start_timestamp'], 
                               periods=row['series_length'], 
                               freq=row['frequency'])
    std_df = pd.DataFrame(index=date_range)
    # we fill NA values with NO_HOLIDAY because rows before an earlier holiday will be NaN
    holidays = std_df.join(bank_holidays, how='left').fillna('NO_HOLIDAY')
    weather = std_df.join(weather_hourly, how='left')
    assert len(holidays) == row['series_length'], 'Length of holidays should be same as series length'
    assert len(weather)== row['series_length'], 'Length of weather should be same as series length'
    row['holidays'] = holidays['Type'].values
    for col in weather:
        row[col] = weather[col].values
    return row

In [24]:
hhblock_df = hhblock_df.progress_apply(map_holidays_weather, args=(bank_holidays, weather_hourly), axis=1)
hhblock_df.head()

  0%|          | 0/5560 [00:00<?, ?it/s]

,LCLid,start_timestamp,frequency,energy_consumption,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,MAC000002,2012-10-13,30min,"[0.263, 0.2689999999999999, 0.275, 0.256, 0.21...",24144,Std,ACORN-A,Affluent,block_0,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[186.0, nan, 188.0, nan, 190.0, nan, 203.0, na...","[8.78, nan, 8.27, nan, 7.87, nan, 7.89, nan, 7...","[6.28, nan, 6.21, nan, 6.22, nan, 6.76, nan, 6...","[1007.7, nan, 1007.36, nan, 1006.73, nan, 1006...","[7.55, nan, 7.34, nan, 6.75, nan, 6.89, nan, 7...","[2.28, nan, 1.81, nan, 1.95, nan, 1.83, nan, 1...","[rain, nan, rain, nan, rain, nan, rain, nan, r...","[clear-night, nan, clear-night, nan, partly-cl...","[0.84, nan, 0.87, nan, 0.89, nan, 0.93, nan, 0...","[Clear, nan, Clear, nan, Partly Cloudy, nan, C..."
1,MAC000246,2012-01-01,30min,"[0.509, 0.317, 0.253, 0.249, 0.93, 0.607, 0.10...",37872,Std,ACORN-A,Affluent,block_0,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[229.0, nan, 238.0, nan, 229.0, nan, 231.0, na...","[12.12, nan, 12.59, nan, 12.45, nan, 12.03, na...","[10.97, nan, 11.02, nan, 11.04, nan, 10.94, na...","[1008.1, nan, 1007.88, nan, 1007.95, nan, 1007...","[12.12, nan, 12.59, nan, 12.45, nan, 12.03, na...","[5.9, nan, 6.06, nan, 5.31, nan, 4.68, nan, 4....","[rain, nan, rain, nan, rain, nan, rain, nan, r...","[partly-cloudy-night, nan, cloudy, nan, partly...","[0.93, nan, 0.9, nan, 0.91, nan, 0.93, nan, 0....","[Mostly Cloudy, nan, Overcast, nan, Mostly Clo..."
2,MAC000450,2012-03-23,30min,"[1.337, 1.426, 0.996, 0.971, 0.994, 0.952, 0.8...",33936,Std,ACORN-A,Affluent,block_0,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[78.0, nan, 73.0, nan, 81.0, nan, 80.0, nan, 7...","[8.76, nan, 8.54, nan, 8.09, nan, 7.34, nan, 7...","[7.25, nan, 7.12, nan, 7.17, nan, 6.68, nan, 6...","[1027.41, nan, 1026.91, nan, 1026.54, nan, 102...","[7.59, nan, 7.43, nan, 7.24, nan, 7.34, nan, 7...","[2.18, nan, 2.07, nan, 1.72, nan, 1.34, nan, 1...","[rain, nan, rain, nan, rain, nan, rain, nan, r...","[fog, nan, fog, nan, fog, nan, fog, nan, fog, ...","[0.9, nan, 0.91, nan, 0.94, nan, 0.96, nan, 0....","[Foggy, nan, Foggy, nan, Foggy, nan, Foggy, na..."
3,MAC001074,2012-05-09,30min,"[0.18, 0.086, 0.106, 0.173, 0.146, 0.223, 0.21...",31680,ToU,ACORN-,ACORN-,block_0,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[215.0, nan, 207.0, nan, 215.0, nan, 216.0, na...","[11.46, nan, 11.38, nan, 11.38, nan, 10.86, na...","[10.23, nan, 10.17, nan, 10.24, nan, 10.13, na...","[1007.39, nan, 1007.21, nan, 1007.06, nan, 100...","[11.46, nan, 11.38, nan, 11.38, nan, 10.86, na...","[2.35, nan, 2.15, nan, 1.84, nan, 1.22, nan, 0...","[rain, nan, rain, nan, rain, nan, rain, nan, r...","[partly-cloudy-night, nan, partly-cloudy-night...","[0.92, nan, 0.92, nan, 0.93, nan, 0.95, nan, 0...","[Partly Cloudy, nan, Mostly Cloudy, nan, Mostl..."
4,MAC003223,2012-09-18,30min,"[0.076, 0.079, 0.123, 0.109, 0.051, 0.069, 0.0...",25344,Std,ACORN-A,Affluent,block_0,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[236.0, nan, 240.0, nan, 242.0, nan, 244.0, na...","[14.06, nan, 13.12, nan, 12.53, nan, 12.07, na...","[10.82, nan, 10.29, nan, 9.86, nan, 9.83, nan,...","[1011.09, nan, 1010.82, nan, 1010.65, nan, 101...","[14.06, nan, 13.12, nan, 12.53, nan, 12.07, na...","[3.86, nan, 3.81, nan, 4.27, nan, 4.12, nan, 4...","[rain, nan, rain, nan, rain, nan, rain, nan, r...","[clear-night, nan, clear-night, nan, clear-nig...","[0.81, nan, 0.83, nan, 0.84, nan, 0.86, nan, 0...","[Clear, nan, Clear, nan, Clear, nan, Partly Cl..."


### Saving the combined data to disk

Split the DataFrame into chunks and save these chunks as parquet files. This has the advantages:

1. leverages the compression that comes with the format
2. reading in parts of the whole data allows for quick iteration and experimentation
3. data types are retained between read and write operations

In [25]:
os.makedirs('../data/preprocessed', exist_ok=True)

We save a map of which households belong to each Acorn group as a pickle file. This will help us read in households for each of these groups straight from our Parquet files later on.

In [26]:
# save the LCLid to Acorn group map as a pickle to be used later
hhblock_df[['LCLid', 'file', 'Acorn_grouped']].to_pickle(f'../data/preprocessed/london_smart_meters_lclid_acorn_map.pkl')

In [27]:
# split the blocks into 8 chunks
blocks = [f'block_{i}' for i in range(111)]

n_chunks= 8
split_blocks = [blocks[i:i + n_chunks] for i in range(0, len(blocks), n_chunks)] 

In [30]:
#Writing each chunk to disk
for blk in tqdm(split_blocks):
    df = hhblock_df.loc[hhblock_df.file.isin(blk)]
    blk = [int(b.replace('block_', '')) for b in blk]
    block_str = f'block_{min(blk)}-{max(blk)}'
    df.to_parquet(f'../data/preprocessed/london_smart_meters_merged_{block_str}.parquet')

  0%|          | 0/14 [00:00<?, ?it/s]